In [1]:
import pandas as pd             # data package
import matplotlib.pyplot as plt # graphics 
import datetime as dt
import numpy as np

import requests, io             # internet and input tools  
import zipfile as zf            # zip file tools 
import os
import geopandas as gpd # this is the main geopandas 

import pyarrow as pa
import pyarrow.parquet as pq
 
from bokeh.palettes import brewer, Spectral6
from bokeh.io import show, output_file, curdoc
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool, Panel, Tabs, GeoJSONDataSource, LinearColorMapper
from bokeh.models import ColorBar
from bokeh.layouts import column, gridplot, row
from bokeh.transform import factor_cmap

In [2]:
crl = ["forestgreen","darkslategray","mediumblue","crimson"]

In [3]:
df = pd.read_csv(".//data/exports_by_naics.csv")

In [4]:
df["high_cat"] = df.naics3.astype(str).str[0]

In [5]:
df["time"] = pd.to_datetime(df.time)

df.set_index(["time","naics3"],inplace = True)

In [6]:
naics_cat = list(df.loc["2018-01-01"].high_cat.unique())

color_map_naics = dict(zip(naics_cat,crl))

name_map = dict(zip(naics_cat,["Agriculture", "Oil/Mining","Manufacturing", "N/S"]))

In [7]:
df["colors"] = df.high_cat.map(color_map_naics)

df["name"] = df.high_cat.map(name_map)

In [8]:
df.rename(mapper = {"china_trade_y":"weight"}, inplace = True, axis = 1)

In [9]:
#df.set_index(["time","naics3"],inplace = True)

df.head()

tariff_trd_w_avg   total_trade   china_trade high_cat  \
time       naics3                                                          
2015-01-01 111             7.731278  6.628535e+09  2.260464e+09        1   
           112            11.923828  1.415040e+08  3.176711e+06        1   
           113             0.173604  1.617550e+08  5.432390e+07        1   
           114             8.184240  1.774655e+08  3.002359e+07        1   
           211             0.656623  1.728807e+09  2.445570e+07        2   

                          colors         name  
time       naics3                              
2015-01-01 111       forestgreen  Agriculture  
           112       forestgreen  Agriculture  
           113       forestgreen  Agriculture  
           114       forestgreen  Agriculture  
           211     darkslategray   Oil/Mining

In [10]:
def load_map():
    
    map_projection = "epsg:2163"
    
    county_shape = ".\\shapefiles\\lake\\ne_10m_lakes.shx"

    lake_map = gpd.read_file(county_shape)

    lake_map = lake_map.to_crs({'init': map_projection})
    
##########################################################################
    
    county_shape = ".\\shapefiles\\land\\ne_50m_land.shx"

    land_map = gpd.read_file(county_shape)

    land_map = land_map.to_crs({'init': map_projection})

    land_map = land_map.iloc[0:1200]
    
##########################################################################
    
    county_shape = ".\\shapefiles\\county\\tl_2017_us_county.shx"

    us_map = gpd.read_file(county_shape)

    us_map = us_map.to_crs({'init': map_projection})
    
    us_map["geometry"] = us_map["geometry"].simplify(4000)
    
    us_map["area_fips"] = (us_map.STATEFP.astype(str) + us_map.COUNTYFP.astype(str)).astype(int)

    us_map.set_index("STATEFP", inplace = True)

    drop_list = ["02","15","72"]

    us_map.drop(drop_list, inplace = True)
    
    us_map.reset_index(inplace = True)
    
##########################################################################
    
    state_shape = ".\\shapefiles\\state\\tl_2017_us_state.shx"

    state_map = gpd.read_file(state_shape)

    state_map = state_map.to_crs({'init': map_projection})

    state_map["geometry"] = state_map["geometry"].simplify(4000)

    state_map.set_index("STATEFP", inplace = True)

    drop_list = ["02","15","72","78","69","66","60",]

    state_map.drop(drop_list, inplace = True)
    
    state_map.reset_index(inplace = True)
    
    state_fp_dict = dict(zip(state_map.STATEFP, state_map.STUSPS))
    
    us_map["STSPS"] = us_map["STATEFP"].map(state_fp_dict)
    
    us_map["NAME"] = us_map["NAME"] + " County, " + us_map["STSPS"]
    
##########################################################################
    
    us_map = gpd.overlay(us_map, land_map,  how='intersection')
    
    great_lakes = ["Lake Superior", "Lake Michigan", "Lake Erie","Lake Superior""Lake Huron"]

    us_map = gpd.overlay(us_map, lake_map[lake_map.name.isin(great_lakes)],  how='difference')
    
    state_map = gpd.overlay(state_map, land_map,  how='intersection')

    state_map = gpd.overlay(state_map, lake_map[lake_map.name.isin(great_lakes)],  how='difference')
    
    return us_map, state_map

In [11]:
##########################################################################
def make_source(time):
    
    datasource = df.loc[time].copy()
    
    datasource.reset_index(inplace = True)
    
    datasource["position"] = datasource.index.values
    
    source = ColumnDataSource(datasource)
    
    return source

In [12]:
def make_tariff_source(time):

    df = pq.read_table(".\\data\\trade_employment_blssingle2020.parquet").to_pandas()

    df["time"] = pd.to_datetime(df.time)

    df.set_index(["area_fips", "time"],inplace = True)

    tariff = df.xs(time, level=1).copy()

    tariff["fips_code"] = tariff.index

    tariff["fips_code"] = tariff["fips_code"].astype(int)
    
    tariff["pop_label"] = tariff["2017_population"].map('{:,.0f}'.format)

    tariff["tariff_label"] = tariff["tariff"].map('{:,.2f}'.format)
    
    return tariff

In [24]:
def make_tradewar_fig(trade_war_time):

    trade_war_time = trade_war_time
    width = 500
    height = 245
##########################################################################
#output_file("bars.html")


    p = figure(plot_height=height, plot_width = width, title="Tariff Rate by NAICs Industry Code",
               toolbar_location = 'below',
           tools = "box_zoom, reset")
    
    source = make_source(trade_war_time)

    p.vbar(x = "position", top = "tariff_trd_w_avg", width = 0.9, color = "colors", alpha = 0.5,
       hatch_pattern = " ",hatch_alpha = 0.10,
       source = source, legend_field=  "name")
    
    mfn_time = "2018-01-01"
    source = make_source(mfn_time)
    
    if dt.datetime.strptime("2018-03-01", "%Y-%m-%d") >= dt.datetime.strptime(trade_war_time, "%Y-%m-%d"):
        alpha_tw = 0
    else:
        alpha_tw = 0.25

    p.vbar(x = "position", top = "tariff_trd_w_avg", width = 0.9, alpha = alpha_tw,
       hatch_pattern = ">",hatch_alpha = 0.10,
       source = source, color = "colors")
##########################################################################33

    TOOLTIPS = """
        <div style="background-color:#F5F5F5; opacity: 0.95;">
            <div style = "text-align:left;">
                <span style="font-size: 10px; font-weight: bold;">Industry Code: @naics3</span>
            </div>
            <div style = "text-align:left;">
                <span style="font-size: 10px; font-weight: bold">Tariff: @tariff_trd_w_avg</span>
            </div>
        </div>
    """

    p.add_tools(HoverTool(tooltips = TOOLTIPS))
##########################################################################
    p.xgrid.grid_line_color = None
    p.y_range.start = 0
    p.y_range.end = 45
    p.x_range.start = -0.75
    p.x_range.end = 30
    p.toolbar.autohide = True

    p.xaxis.major_tick_line_color = None  # turn off x-axis major ticks
    p.xaxis.minor_tick_line_color = None  # turn off x-axis minor ticks
    p.xaxis.major_label_text_font_size = '0pt'  # turn off x-axis tick labels
    p.yaxis.minor_tick_line_color = None
    
    p.border_fill_color = "#F5F5F5"    
    p.legend.orientation = "horizontal"
    
    p.background_fill_color = "#F5F5F5" 
    p.background_fill_alpha = 0.75
##########################################################################
    start = "2017-08-01"

    tariffs_over_time = df.groupby(["time"]).agg({"tariff_trd_w_avg": "mean"})

    source = ColumnDataSource(tariffs_over_time.loc[start:trade_war_time].reset_index())
    
    avgtar = str(round(tariffs_over_time.loc[start:trade_war_time].iloc[-1][0],2))
    
    ptime = figure(plot_height=height, plot_width = width, x_axis_type="datetime", 
                   title="Average Tariff Rate:" + avgtar, toolbar_location = 'below',
           tools = "box_zoom, reset")

    ptime.line(x = "time", y = "tariff_trd_w_avg", line_width=3, color='darkblue',source = source)

    source = ColumnDataSource(tariffs_over_time.loc[start:].reset_index())

    ptime.line(x = "time", y = "tariff_trd_w_avg", line_width=3, color='gray',source = source, alpha = 0.20)

    ptime.vbar(x = dt.datetime.strptime("2018-04-01","%Y-%m-%d"), 
           color='gray', top = 50, width = 0.1, alpha = 0.20)

    ptime.vbar(x = dt.datetime.strptime("2018-07-01","%Y-%m-%d"), 
           color='gray', top = 50, width = 0.1, alpha = 0.20)

    ptime.vbar(x = dt.datetime.strptime("2018-08-23","%Y-%m-%d"), 
           color='gray', top = 50, width = 0.1, alpha = 0.20)

    ptime.vbar(x = dt.datetime.strptime("2018-09-24","%Y-%m-%d"), 
           color='darkgray', top = 50, width = 0.1, alpha = 0.20)
##########################################################################
    TIMETOOLTIPS = """
    <div style="background-color:#F5F5F5; opacity: 0.95; border: 0px 0px 0px 0px">
        <div style = "text-align:left;">
            <span style="font-size: 10px; font-weight: bold">Tariff: @tariff_trd_w_avg</span>
        </div>
    </div>
    """

    ptime.add_tools(HoverTool(tooltips = TIMETOOLTIPS))

    ptime.y_range.start = 0
    ptime.y_range.end = 25
    ptime.x_range.start = dt.datetime.strptime(start,"%Y-%m-%d")
    ptime.x_range.end = dt.datetime.strptime("2020-01-01","%Y-%m-%d")
    ptime.toolbar.autohide = True
    ptime.yaxis.minor_tick_line_color = None  # turn off y-axis minor ticks
    ptime.border_fill_color = "#F5F5F5"
    
    ptime.background_fill_color = "#F5F5F5" 
    ptime.background_fill_alpha = 0.75
##########################################################################

    pmap = make_map(trade_war_time)

##########################################################################
    #outfig = column(p, ptime)
    #outfig = layout([[ptime, p],[pmap]])
    outfig = row(pmap, column(ptime, p))
    #show(outfig)
    return outfig

In [14]:
us_map_fixed, state_map = load_map()


In [ ]:
tab1 = Panel(child= make_tradewar_fig("2018-03-01"), title="Pre Trade War")

tab2 = Panel(child= make_tradewar_fig("2018-04-01"), title="232 Retaliation")

tabs = Tabs(tabs=[ tab1, tab2])

show(tabs)

In [20]:
#us_map_fixed, state_map = load_map()

output_file('.\\docs\\trade_tracker.html')

tab1 = Panel(child= make_tradewar_fig("2018-03-01"), title="Pre Trade War")

tab2 = Panel(child= make_tradewar_fig("2018-04-01"), title="232 Retaliation")

tab3 = Panel(child= make_tradewar_fig("2018-07-01"), title="301 Retaliation, Phase 1")

tab4 = Panel(child= make_tradewar_fig("2018-09-01"), title="Phase 2")

tab5 = Panel(child= make_tradewar_fig("2018-10-01"), title="Phase 3")

tab6 = Panel(child= make_tradewar_fig("2019-06-01"), title="Talks Fail")

tab7 = Panel(child= make_tradewar_fig("2019-09-01"), title="More Retaliation")

#tab7 = Panel(child= make_tradewar_fig("2020-01-01"), title="Peace?")

tabs = Tabs(tabs=[ tab1, tab2, tab3, tab4, tab5, tab6, tab7 ])

show(tabs)

In [22]:
def make_map(time):

    tariff_df = make_tariff_source(time)
    
    
    
    timedt = dt.datetime.strptime(time, "%Y-%m-%d")
    
    month = timedt.strftime("%B")
    year = timedt.strftime("%Y")
    
    tariff_df.tariff.fillna(0, inplace = True)
    
    var = ["tariff","fips_code","pop_label","tariff_label"]
    
    us_map = us_map_fixed.merge(tariff_df[var], left_on='area_fips',
                      right_on = "fips_code", how = "inner", indicator = True)
    ################################################################################
        
    title = "Tariff Exposure by County as of: " + month + " " + year
        
    state_geosource = GeoJSONDataSource(geojson = state_map.to_json())

    geosource = GeoJSONDataSource(geojson = us_map.to_json())
    
    palette = brewer['RdBu'][10]
    
    color_mapper = LinearColorMapper(palette = palette, low = 0, high = 10)

    color_bar = ColorBar(color_mapper = color_mapper, 
                     label_standoff = 8,
                     width = 20, height = 420,
                     border_line_color = None,
                     orientation = "vertical",
                     location=(0,0))

# Create figure object.
    p = figure( 
           plot_height = 490 ,
           plot_width = 750, 
           toolbar_location = 'below',
           tools = "box_zoom, reset",
            title = title)
    
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None

    states = p.patches('xs','ys', source = geosource,
                   fill_color = {"field" :'tariff',
                                 "transform" : color_mapper},
                   line_color = "gray", 
                   line_width = 0.25, 
                   fill_alpha = 1)

    state_line = p.multi_line('xs','ys', source = state_geosource,
                   line_color = "black", 
                   line_width = 0.5)

    p.axis.visible = False
    p.background_fill_color = "grey"
    p.background_fill_alpha = 0.25
    
    TOOLTIPS = """
    <div style="background-color:#F5F5F5; opacity: 0.95;">
        <div style = "text-align:center;">
            <span style="font-size: 10px; font-weight: bold;">@NAME
        </div>
        <div style = "text-align:left;">
            <span style="font-size: 10px; font-weight: bold">Population: @pop_label &nbsp &nbsp Tariff: @tariff_label</span>
        </div>
    </div>
    """
    p.add_tools(HoverTool(renderers = [states],
                      tooltips = TOOLTIPS))
    p.toolbar.autohide = True
    p.add_layout(color_bar, "left")
    p.border_fill_color = "#F5F5F5"
    color_bar.background_fill_color = "#F5F5F5"
    p.background_fill_color = "#F5F5F5" 
    p.background_fill_alpha = 1.0
    
    #show(p)
    return p

In [158]:
p = make_map("2019-01-01")